In [56]:
from models import gmm, lstm
from extractor import extractor
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from collections import OrderedDict

In [19]:

data_wav = "/Users/oosv/data/voxforge" #Path to .wav file data
languages = ["english", "spanish"]
TOTAL_FRAMES = 150
ENERGY_MIN = 12
DELTA_DIM = 3
CEPSTRAL_COEF = 13

extractor.total_frames = TOTAL_FRAMES
extractor.energy_threshold = ENERGY_MIN
files = []
data_y = []
#num_samples = 100
#get the path for num_samples .wav files
for i,l in enumerate(languages):
    path = os.path.join(data_wav, l)
    for f in os.listdir(path):
        files.append(os.path.join(path, f))
    data_y += [i] * len(os.listdir(path))

print(len(data_y))
data_x , rejections = np.array(extractor.make_feature_set(files))
print(len(data_x), np.shape(data_x[0]))

92408


In [5]:
shape_x, shape_y = np.shape(data_x), np.shape(data_y)
data_x = np.reshape(data_x, [shape_x[0], shape_x[1], shape_x[2] * shape_x[3]])

#data_y = np.reshape(data_y, [shape_y[0], shape_y[1], shape_y[2] * shape_y[3]])

In [7]:
def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [6]:
def remove_rejections(data_y, rejections):
    for rejection in rejections:
        del data_y[rejection]
    return data_y

In [21]:
# Remove Rejections
data_y = remove_rejections(data_y, rejections)

In [22]:
# number of samples per language
counts = Counter(data_y)
print(counts)

Counter({0: 68730, 1: 23470})


In [24]:
# Ensure number of english and spanish samples are equal
data_y_ = np.concatenate((data_y[:counts[1]],data_y[counts[0]:]), axis=0)
data_x_ = np.concatenate((data_x[:counts[1]],data_x[counts[0]:]), axis=0)

print(np.shape(data_x_), np.shape(data_y_))

(46940, 75, 39) (46940,)


In [28]:
# Training models on equal amounts of data
data_x = data_x_
data_y = data_y_
shuffle_in_unison(data_x, data_y)
shape_x = np.shape(data_x)

In [29]:
train_x = data_x[int(shape_x[0]/4):]
test_x = data_x[:int(shape_x[0]/4)]
train_y = data_y[int(shape_x[0]/4):]
test_y = data_y[:int(shape_x[0]/4)]

In [30]:
print(np.shape(train_x), np.shape(train_y))
print(np.shape(test_x), np.shape(test_y))

(35205, 75, 39) (35205,)
(11735, 75, 39) (11735,)


In [32]:
with open('train_x.npy', 'wb') as file:
    np.save(file, train_x)
with open('train_y.npy', 'wb') as file:
    np.save(file, train_y)
with open('test_x.npy', 'wb') as file:
    np.save(file, test_x)
with open('test_y.npy', 'wb') as file:
    np.save(file, test_y)
    

0.5025990626331487

In [72]:
importlib.reload(lstm)
_lstm = lstm.MixedLSTM(
                n_features = 39,
                n_hidden = 2560,
                languages = 2, 
                total_frames = 150,
                dropout=0,
                bidirectional=False,
                lstm_layers = 1,
                linear_layers = 4)

l = torch.load('/Users/oosv/Downloads/model (12)/model_2.pth')
d = OrderedDict()
for k1,k2 in zip(_lstm.state_dict(), l):
    d[k1] = l[k2]

_lstm.load_state_dict(d)
# _train_data = torch.tensor(train_data)
# _train_labels = torch.tensor(train_labels)
# print(_train_labels.size())

In [73]:
pred_x  = _lstm.predict_all(test_x)

In [74]:
# Google
np.average(pred_x == test_y)

0.533958244567533

In [79]:
# Basic version trained on 16xlarge
importlib.reload(lstm)
_lstm = lstm.MixedLSTM(
                n_features = 39,
                n_hidden = 512,
                languages = 2, 
                total_frames = 150,
                dropout=0,
                bidirectional=True,
                lstm_layers = 1,
                linear_layers = 1)

l = torch.load('/Users/oosv/Downloads/model (24)/model_5.pth')
d = OrderedDict()
for k1,k2 in zip(_lstm.state_dict(), l):
    d[k1] = l[k2]

_lstm.load_state_dict(d)
pred_x  = _lstm.predict_all(test_x)
np.average(pred_x == test_y)

KeyboardInterrupt: 

In [ ]:
# Dropout trained on 16xLarge
importlib.reload(lstm)
_lstm = lstm.MixedLSTM(
                n_features = 39,
                n_hidden = 512,
                languages = 2, 
                total_frames = 150,
                dropout=0.5,
                bidirectional=False,
                lstm_layers = 2,
                linear_layers = 1)

l = torch.load('/Users/oosv/Downloads/model (25)/model_5.pth')
d = OrderedDict()
for k1,k2 in zip(_lstm.state_dict(), l):
    d[k1] = l[k2]

_lstm.load_state_dict(d)
pred_x  = _lstm.predict_all(test_x)
np.average(pred_x == test_y)

In [ ]:
# Simple LSTM trained on 8xlarge
importlib.reload(lstm)
_lstm = lstm.MixedLSTM(
                n_features = 39,
                n_hidden = 512,
                languages = 2, 
                total_frames = 150,
                dropout=0.0,
                bidirectional=False,
                lstm_layers = 1,
                linear_layers = 1)

l = torch.load('/Users/oosv/Downloads/model (20)/model_5.pth')
d = OrderedDict()
for k1,k2 in zip(_lstm.state_dict(), l):
    d[k1] = l[k2]

_lstm.load_state_dict(d)
pred_x  = _lstm.predict_all(test_x)
np.average(pred_x == test_y)

In [ ]:
# Simple BiLSTM trained on 16xlarge
importlib.reload(lstm)
_lstm = lstm.MixedLSTM(
                n_features = 39,
                n_hidden = 512,
                languages = 2, 
                total_frames = 150,
                dropout=0.0,
                bidirectional=True,
                lstm_layers = 1,
                linear_layers = 1)

l = torch.load('/Users/oosv/Downloads/model (26)/model_5.pth')
d = OrderedDict()
for k1,k2 in zip(_lstm.state_dict(), l):
    d[k1] = l[k2]

_lstm.load_state_dict(d)
pred_x  = _lstm.predict_all(test_x)
np.average(pred_x == test_y)